<a href="https://colab.research.google.com/github/ashutosh-linux/aashu/blob/main/GEN_AI_COURSEPROJECT__36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Conv2DTranspose, BatchNormalization, ReLU, Reshape
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import cv2
import librosa
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, mean_squared_error, f1_score


In [36]:

# 🔹 Load Dataset (Modify path as needed)
df = pd.read_csv("/movies_youtube_sentiments.csv")

# 🔹 Handle Missing Values & Convert Everything to String
df.fillna("Unknown", inplace=True)

# 🔹 Combine relevant columns for better text input
df['combined_text'] = df[['name', 'genre', 'director', 'writer', 'star']].astype(str).agg(' '.join, axis=1)



<ipython-input-36-17b1ca40553d>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)


In [50]:
def preprocess_text(texts, vocab_size=5000, max_length=50):
    """Tokenize and pad text input"""
    tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
    tokenizer.fit_on_texts(texts)  # The input 'texts' is already a list
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences, tokenizer

In [38]:
def extract_video_features(video_path, frame_rate=1):
    """Extract keyframes from video at a specified frame rate"""
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.resize(frame, (64, 64)))  # Resize for model input
    cap.release()
    return np.array(frames)



In [39]:
def extract_audio_features(audio_path, sr=22050, n_mfcc=13):
    """Extract MFCC features from audio"""
    y, sr = librosa.load(audio_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc, axis=1)



In [40]:
def build_text_encoder(vocab_size, embedding_dim=128, lstm_units=256):
    """Text Encoder for user input (Genre, Storyline, etc.)"""
    text_input = Input(shape=(None,))  # Input is sequence of word indices
    x = Embedding(vocab_size, embedding_dim)(text_input)
    x = LSTM(lstm_units)(x)
    return Model(text_input, x, name="Text_Encoder")



In [43]:
def build_video_generator(latent_dim=256):
    """Video Generator using GAN approach"""
    noise_input = Input(shape=(latent_dim,))
    x = Dense(8*8*512, activation='relu')(noise_input)
    x = Reshape((8, 8, 512))(x)
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(x) # Output: (16, 16, 256)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x) # Output: (32, 32, 128)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)  # Output: (64, 64, 64)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # Changed strides to 1 to get (64, 64, 3) output
    video_output = Conv2DTranspose(3, kernel_size=3, strides=1, padding='same', activation='tanh')(x)
    return Model(noise_input, video_output, name="Video_Generator")



In [44]:
def build_movie_generation_model(vocab_size, latent_dim=256):
    """Complete Model: Combines Text Input and Video Generation"""
    text_encoder = build_text_encoder(vocab_size)
    video_generator = build_video_generator(latent_dim)

    user_input = Input(shape=(None,))
    text_features = text_encoder(user_input)
    generated_video = video_generator(text_features)

    return Model(user_input, generated_video, name="Movie_Generation_Model")

In [45]:
# 🔹 Define Model Parameters
vocab_size = 5000  # Adjust based on dataset
latent_dim = 256
movie_model = build_movie_generation_model(vocab_size, latent_dim)
movie_model.summary()

# 🔹 Preprocess dataset with updated 'combined_text'
processed_text, tokenizer = preprocess_text(df['combined_text'])
df['processed_text'] = processed_text.tolist() # Convert 2D array to a list of lists

print("✅ Data preprocessing complete. Ready for training!")

# 🔹 Example: Generate fake labels for training (Modify based on real data)
X_train = np.array(df['processed_text'].tolist()) # Convert to NumPy array with correct dtype
y_train = np.random.rand(len(X_train), 64, 64, 3)  # Dummy video output data

# 🔹 Train the Model
movie_model.compile(optimizer='adam', loss='mse')
movie_model.fit(X_train, y_train, epochs=5, batch_size=16)

print("✅ Model training complete!")

# 🔹 Evaluation: Compute Metrics
y_pred = movie_model.predict(X_train)

# Flatten for comparison
y_train_flat = y_train.flatten()
y_pred_flat = y_pred.flatten()

accuracy = accuracy_score(y_train_flat.round(), y_pred_flat.round())
precision = precision_score(y_train_flat.round(), y_pred_flat.round(), average='macro')
mse = mean_squared_error(y_train_flat, y_pred_flat)
f1 = f1_score(y_train_flat.round(), y_pred_flat.round(), average='macro')

print(f"🔹 Accuracy: {accuracy}")
print(f"🔹 Precision: {precision}")
print(f"🔹 MSE: {mse}")
print(f"🔹 F1 Score: {f1}")

print("🎬 AI Movie Trailer Generation Pipeline Complete! 🚀")


Model: "Movie_Generation_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_65 (InputLayer)          │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Text_Encoder (Functional)            │ (None, 256)                 │       1,034,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Video_Generator (Functional)         │ (None, 64, 64, 3)           │       9,973,635 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,007,875 (41.99 MB)

 Trainable params: 11,006,979 (41.99 MB)

 Non-trainable params: 896 (3.50 KB)

✅ Data preprocessing complete. Ready for training!
Epoch 1/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 71s 916ms/step - loss: 0.2237
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 68s 960ms/step - loss: 0.0873
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 78s 908ms/step - loss: 0.0858
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 69s 980ms/step - loss: 0.0848
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 908ms/step - loss: 0.0844
✅ Model training complete!
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 510ms/step
🔹 Accuracy: 0.5018351421097285
🔹 Precision: 0.5020092513388199
🔹 MSE: 0.08491004214301275
🔹 F1 Score: 0.4871011941668939
🎬 AI Movie Trailer Generation Pipeline Complete! 🚀


In [47]:
def build_optimized_video_generator(latent_dim=256):
    noise_input = Input(shape=(latent_dim,))
    x = Dense(8*8*512, activation='relu')(noise_input)
    x = Reshape((8, 8, 512))(x)
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.3)(x)  # Prevent Overfitting
    x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.3)(x)
    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # Changed strides to 1 to match the target data shape (64, 64, 3)
    video_output = Conv2DTranspose(3, kernel_size=3, strides=1, padding='same', activation='tanh')(x)
    return Model(noise_input, video_output, name="Optimized_Video_Generator")


In [48]:
# 🔹 Update Movie Generation Model
def build_optimized_movie_model(vocab_size, latent_dim=256):
    text_encoder = build_text_encoder(vocab_size)
    video_generator = build_optimized_video_generator(latent_dim)

    user_input = Input(shape=(None,))
    text_features = text_encoder(user_input)
    generated_video = video_generator(text_features)

    return Model(user_input, generated_video, name="Optimized_Movie_Generation_Model")

# 🔹 Train Optimized Model
optimized_movie_model = build_optimized_movie_model(vocab_size)
optimized_movie_model.compile(optimizer=Adam(learning_rate=0.0001, clipnorm=1.0), loss='mse')  # Gradient Clipping
optimized_movie_model.fit(X_train, y_train, epochs=10, batch_size=16)

print("✅ Model Optimization Complete!")


Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 77s 998ms/step - loss: 0.5652
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 80s 965ms/step - loss: 0.1268
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 68s 979ms/step - loss: 0.1088
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - loss: 0.1025
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - loss: 0.0987  
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 70s 993ms/step - loss: 0.0967
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - loss: 0.0949
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - loss: 0.0931
Epoch 9/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - loss: 0.0918
Epoch 10/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - loss: 0.0908
✅ Model Optimization Complete!


In [51]:
def generate_movie_trailer(user_input_text):
    # 🔹 Convert user input into model-friendly format
    user_sequence, _ = preprocess_text([user_input_text])

    # 🔹 Generate video using AI model
    generated_video = optimized_movie_model.predict(user_sequence)

    # 🔹 Save the generated video
    video_filename = "generated_trailer.mp4"
    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'mp4v'), 10, (64, 64))

    for frame in generated_video[0]:
        frame = (frame * 255).astype(np.uint8)  # Convert to valid image format
        out.write(frame)

    out.release()

    print(f"🎬 Trailer Generated! Saved as {video_filename}")

# 🔹 Example Usage
user_input = "A sci-fi thriller about an AI revolution in 2099."
generate_movie_trailer(user_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
🎬 Trailer Generated! Saved as generated_trailer.mp4


In [55]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/generate_trailer', methods=['POST'])
def generate_trailer():
    data = request.get_json()
    user_input = data.get("description", "")

    if not user_input:
        return jsonify({"error": "Please provide a movie description"}), 400

    generate_movie_trailer(user_input)

    return jsonify({"message": "Trailer generated!", "file": "generated_trailer.mp4"})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [53]:
!pip install streamlit opencv-python-headless numpy tensorflow librosa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00
